In [1]:
%load_ext blackcellmagic

# Data Base Analysis

In [2]:
import sqlalchemy as sa
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy.dialects import oracle
from sqlalchemy.sql import select
from sqlalchemy.sql import text

import psycopg2
import mysql.connector as msql
import cx_Oracle

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

import dash
from dash import dash_table
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html
import dash_cytoscape as cyto
from jupyter_dash import JupyterDash
from dash import no_update
from dash.dependencies import Input, Output, State

import networkx as nx
import time
from tqdm import tqdm
import json
import os

/Users/elenamylonas/opt/anaconda3/lib/python3.8/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


## Functions

In [3]:
def query_w_header(query):
    with engine.connect() as con:
        statement = text(query)
        res = con.execute(statement)
        df = pd.DataFrame(res)

    return df


def query_wo_header(query):
    with engine.connect() as con:

        statement = text(query)
        df = pd.read_sql(statement, con)

    return df


def ego_graph(G, n, radius=1, center=True, undirected=False, distance=None):
    if undirected:
        if distance is not None:
            sp, _ = nx.single_source_dijkstra(
                G.to_undirected(), n, cutoff=radius, weight=distance
            )
        else:
            sp = dict(
                nx.single_source_shortest_path_length(
                    G.to_undirected(), n, cutoff=radius
                )
            )
    else:
        if distance is not None:
            sp, _ = nx.single_source_dijkstra(G, n, cutoff=radius, weight=distance)
        else:
            sp = dict(nx.single_source_shortest_path_length(G, n, cutoff=radius))

    H = G.subgraph(sp).copy()
    if not center:
        H.remove_node(n)
    return H

## DB connection

In [4]:
try:
    with open("config1.json") as json_data_file:
        data = json.load(json_data_file)
except FileNotFoundError:
        print('config.json file missing')

In [5]:
connector = data["database"]["connector"]
sqluser = data["database"]["user"]
sqlpass = data["database"]["passwd"]
host = data["database"]["host"]
port = data["database"]["port"]
dbname = data["database"]["db"]
database = data["database"]["database"]
driver = data["database"]["driver"]
schema_name = data["database"]["schema"]

connection = "{}+{}://{}:{}@{}:{}/{}".format(
    driver, connector, sqluser, sqlpass, host, port, dbname
)
engine = create_engine(connection)

## Queries SQL
#### All queries needed to run these notebook

In [6]:
if database == "postgres":

    # Query1: Discover schemas
    query1_schemas = open("sql_queries/postgres/query1.sql").read()

    # Query2: Show all tables and columns

    query2_all_tables = open("sql_queries/postgres/query2.sql").read()

    # Query3: All keys (primary and foreigner), and target tables

    query3_all_keys = open("sql_queries/postgres/query3.sql").read()

    # Query4: First 10 lines from each table

    query4_first10 = open("sql_queries/postgres/query4.sql").read()
    # Query5: Columns Stats

    query5_col_stats = open("sql_queries/postgres/query5.sql").read()

    # Query6: Columns commentaries

    query6_col_comments = open("sql_queries/postgres/query6.sql").read()

    # Query7: Tables commentaries - number of rows
    query7_table_comments = open("sql_queries/postgres/query7.sql").read()



In [7]:
if database == "oracle":

    # Query1: Discover schemas

    query1_schemas = open("sql_queries/oracle/query1.sql").read()

    # Query2: Show all tables and columns

    query2_all_tables = open("sql_queries/oracle/query2.sql").read()
    
    # Query3: All keys (primary and foreigner), and target tables

    query3_all_keys = open("sql_queries/oracle/query3.sql").read()

    # Query4: First 10 lines from each table

    query4_first10 = open("sql_queries/oracle/query4.sql").read()

    # Query5: Columns Stats

    query5_col_stats = open("sql_queries/oracle/query5.sql").read()

    # Query6: Columns commentaries

    query6_col_comments = open("sql_queries/oracle/query6.sql").read()

    # Query7: Tables commentaries - number of rows

    query7_table_comments = open("sql_queries/oracle/query7.sql").read()

In [8]:
if database == "mysql":

    # Query1: Discover schemas

    query1_schemas = open("sql_queries/mysql/query1.sql").read()

    # Query2: Show all tables and columns

    query2_all_tables = open("sql_queries/mysql/query2.sql").read()
    
    # Query3: All keys (primary and foreigner), and target tables

    query3_all_keys = open("sql_queries/mysql/query3.sql").read()

    # Query4: First 10 lines from each table

    query4_first10 = open("sql_queries/mysql/query4.sql").read()

    # Query5: Columns Stats (#Fro MySql incomplete query)

    query5_col_stats = open("sql_queries/mysql/query5.sql").read()

    # Query6: Columns commentaries

    query6_col_comments = open("sql_queries/mysql/query6.sql").read()

    # Query7: Tables commentaries - number of rows

    query7_table_comments = open("sql_queries/mysql/query7.sql").read()

## 1) Discovering DataBase schema

In [9]:
df_schema = list(query_w_header(query1_schemas)[0])

In [10]:
#schema_name = input()
schema_name = data['database']['schema']

## 2) Info schema tables and columns

#### 2.1 Obtain all the tables names and columns from information.schema.
#### Schema name of the data base, example: mimiciii for MIMIC and classicmodels for the toy database (avoid systems folders)

Two data frames are generated:

    1. df_table: containing all table names and columns names (all database)
    2. df_table_pk: containing all table names and columns names of only primary and foreig keys.

In [11]:
if schema_name != "all":

    if database == "postgres":
        query2_all_tables = " ".join(
            [query2_all_tables, """and t.table_schema ='{}'"""]
        ).format(schema_name)
        query3_all_keys = " ".join(
            [query3_all_keys, """where kcu.constraint_schema ='{}'"""]
        ).format(schema_name)
        query5_col_stats = " ".join(
            [query5_col_stats, """and t.table_schema = '{}';"""]
        ).format(schema_name)
        query7_table_comments = " ".join(
            [query7_table_comments, """or u1.schemaname = '{}';"""]
        ).format(schema_name)

    elif database == "mysql":
        query2_all_tables = " ".join(
            [query2_all_tables, """where TABLE_SCHEMA ='{}' """]
        ).format(schema_name)
        query3_all_keys = " ".join(
            [query3_all_keys, """where TABLE_SCHEMA ='{}' """]
        ).format(schema_name)
        query5_col_stats = " ".join(
            [query5_col_stats, """where c.table_schema = '{}' """]
        ).format(schema_name)
        query6_col_comments = " ".join(
            [query6_col_comments, """where table_schema = '{}' """]
        ).format(schema_name)
        query7_table_comments = " ".join(
            [query7_table_comments, """where table_schema = '{}'; """]
        ).format(schema_name)

    elif database == "oracle":
        query2_all_tables = " ".join(
            [query2_all_tables, """where owner = '{}' """]
        ).format(schema_name)
        query3_all_keys = " ".join(
            [query3_all_keys, """AND uc_r.owner = '{}' """]
        ).format(schema_name)
        query5_col_stats = " ".join(
            [query5_col_stats, """WHERE a.owner = '{}' """]
        ).format(schema_name)
        query6_col_comments = " ".join(
            [query6_col_comments, """WHERE a.owner = '{}' """]
        ).format(schema_name)
        query7_table_comments = " ".join(
            [query7_table_comments, """where c.owner = '{}' """]
        ).format(schema_name)

else:
    pass

In [12]:
df_table = query_w_header(query2_all_tables)
df_table.columns = ["schema_name", "table_name", "column_name"]

#### 2.2) If primary and foreig keys are used, the following query allows the selection of only those columns.

In [13]:
try:
    with tqdm(total=1) as progress:
        df_table_pk = query_w_header(query3_all_keys)
        progress.update()
    print(progress.format_interval(progress.format_dict["elapsed"]))
except:
    df_table_pk = pd.DataFrame()

100%|██████████| 1/1 [01:06<00:00, 66.59s/it]

01:06


In [14]:
df_table_pk

,0,1,2,3,4,5
0,mimiciii,admissions,subject_id,admissions_fk_subject_id,patients,subject_id
1,mimiciii,callout,subject_id,callout_fk_subject_id,patients,subject_id
2,mimiciii,callout,hadm_id,callout_fk_hadm_id,admissions,hadm_id
3,mimiciii,chartevents,subject_id,chartevents_fk_subject_id,patients,subject_id
4,mimiciii,chartevents_1,subject_id,chartevents_fk_subject_id,patients,subject_id
...,...,...,...,...,...,...
216371,mimiciii,services,subject_id,services_fk_subject_id,patients,subject_id
216372,mimiciii,services,hadm_id,services_fk_hadm_id,admissions,hadm_id
216373,mimiciii,transfers,subject_id,transfers_fk_subject_id,patients,subject_id
216374,mimiciii,transfers,hadm_id,transfers_fk_hadm_id,admissions,hadm_id


In [15]:
if database == "oracle":
    df_table_pk.columns = [
        "schema_name",
        "table_name",
        "column_name",
        "key",
        "key2",
        "target",
        "target_label",
    ]
    df_table_pk.drop_duplicates(inplace=True, ignore_index=True)
    df_table_pk2 = df_table_pk.dropna(axis=0)
    df_table_pk2.columns = [
        "schema",
        "from",
        "label",
        "key",
        "key2",
        "target",
        "target_label",
    ]
    df_table_pk2.reset_index(inplace=True, drop=True)
    df_table_pk2.drop_duplicates(inplace=True, ignore_index=True)
    df_table_pk.fillna(value=np.nan, inplace=True)

else:
    df_table_pk.columns = [
        "schema_name",
        "table_name",
        "column_name",
        "key2",
        "target",
        "target_label",
    ]
    df_table_pk.drop_duplicates(inplace=True, ignore_index=True)
    df_table_pk2 = df_table_pk.dropna(axis=0)
    df_table_pk2.columns = ["schema", "from", "label", "key", "target", "target_label"]
    df_table_pk2.reset_index(inplace=True, drop=True)
    df_table_pk2.drop_duplicates(inplace=True, ignore_index=True)
    df_table_pk.fillna(value=np.nan, inplace=True)

In [16]:
df_table_pk.head(3)

,schema_name,table_name,column_name,key2,target,target_label
0,mimiciii,admissions,subject_id,admissions_fk_subject_id,patients,subject_id
1,mimiciii,callout,subject_id,callout_fk_subject_id,patients,subject_id
2,mimiciii,callout,hadm_id,callout_fk_hadm_id,admissions,hadm_id


#### 2.3) Obtaining first 10 rows of each table

In [17]:
list_tables = list(df_table.table_name.unique())

In [18]:
dict_schema_table = (
    df_table[["schema_name", "table_name"]].set_index("table_name").T.to_dict("list")
)

<ipython-input-18-7f4014e2e7c0>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  df_table[["schema_name", "table_name"]].set_index("table_name").T.to_dict("list")


## 3) Info columns per table (stats)

#### General stats from all data base, number of tables, number of columns, amount of data.

In [19]:
with tqdm(total=1) as progress:
    df = query_wo_header(query5_col_stats)
    df.columns = [
        "schema_name", 
        "table_name", 
        "column_name", 
        "Dtype", 
        "nb_rows", 
        "null_count"
    ]
    progress.update()
print(progress.format_interval(progress.format_dict["elapsed"]))

100%|██████████| 1/1 [00:19<00:00, 19.21s/it]

00:19


In [20]:
with tqdm(total=1) as progress:
    for i in range(len(df)):
        if df.loc[i,"nb_rows"] != 0:
            df.loc[i, "percentage"] = ((df.loc[i,"nb_rows"]-df.loc[i,"null_count"]) * 100)/ df.loc[i,"nb_rows"]
            progress.update()
        else:
            df.loc[i,'percentage'] = 0
    print(progress.format_interval(progress.format_dict["elapsed"]))

df.fillna(value=np.nan, inplace=True)

3405it [00:00, 6321.65it/s]          

00:00


In [21]:
df.head()

,schema_name,table_name,column_name,Dtype,nb_rows,null_count,percentage
0,mimiciii,admissions,row_id,integer,58976.0,0.0,100.0
1,mimiciii,admissions,subject_id,integer,58976.0,0.0,100.0
2,mimiciii,admissions,hadm_id,integer,58976.0,0.0,100.0
3,mimiciii,admissions,admittime,timestamp without time zone,58976.0,0.0,100.0
4,mimiciii,admissions,dischtime,timestamp without time zone,58976.0,0.0,100.0


## 3.2 Generation of .CSV files    
    1. Column.csv: table name, column name, nb of filled rows, data type, completion %, primary/foreign key, table related (target) , target column name, exmple of data, comments (from information_schema).
    2. Table.csv: names, number of rows, number of columns, primary key, # tables related), comments

#### 3.2.1 Column CSV generation:

In [22]:
def columns_data(df, df_table_pk, query6_col_comments):

    # generate the table backbone merging all columns stats (df) and keys information(df_table_pk)
    columns_csv = pd.merge(
        df, df_table_pk, how="left", on=["schema_name","table_name", "column_name"]
    )

    # obtain the columns commentaries
    try:
        df_table_comments = query_w_header(query6_col_comments)
        df_table_comments.columns = [
            "table_name",
            "column_name",
            "comments",
        ]
    except:
        df_table_comments = pd.DataFrame(
            columns=["table_name", "column_name", "comments"]
        )

    # merge the columns commentaries with table backbone
    columns_csv = columns_csv.merge(
        df_table_comments, how="left", on=["table_name", "column_name"]
    )
    columns_csv.fillna(value=np.nan, inplace=True)

    return columns_csv

In [23]:
columns_csv = columns_data(df, df_table_pk, query6_col_comments)

In [24]:
columns_csv.to_csv("columns_info.csv", index=False)

#### 3.2.2 Table CSV generation:

In [25]:
def table_data(columns, df_table_pk):

    # obtain general table information through columns_csv analysis
    # tables name, nb of columns
    table_name = columns.groupby(["schema_name", "table_name"])["column_name"].count()
    table_name = pd.DataFrame(table_name)
    table_name.reset_index(inplace=True)

    # keys names
    keys_list = (
        columns.groupby(["schema_name", "table_name"])["key2"]
        .unique()
        .apply(list)
        .apply(lambda x: [i for i in x if str(i) != "nan"])
    )
    key_type = pd.DataFrame(keys_list)
    key_type.reset_index(inplace=True)

    # merging table name and nb of columns with key info
    table_csv = table_name.merge(key_type, how="left", on=["schema_name", "table_name"])

    # columns key name
    key_list2 = (
        df_table_pk.groupby(["schema_name", "table_name"])["column_name"]
        .unique()
        .apply(list)
        .apply(lambda x: [i for i in x if str(i) != "nan"])
    )
    key_name = pd.DataFrame(key_list2)
    key_name.reset_index(inplace=True)

    # merging of columns key name
    table_csv = table_csv.merge(key_name, how="left", on=["schema_name", "table_name"])

    # obtaining target tables
    target_table = (
        columns.groupby(["schema_name", "table_name"])["target"]
        .unique()
        .apply(list)
        .apply(lambda x: [i for i in x if str(i) != "nan"])
    )
    target = pd.DataFrame(target_table)
    target.reset_index(inplace=True)

    table_csv = table_csv.merge(target, how="left", on=["schema_name", "table_name"])

    # obtaining table comments
    try:
        table_rows = query_w_header(query7_table_comments)
        table_rows.columns = [
            "schema_name",
            "table_name",
            "table_rows",
            "table_comment",
        ]
    except:
        table_rows = pd.DataFrame(
            columns=["schema_name", "table_name", "table_rows", "table_comment"]
        )

    # merging to final table
    table_csv = table_csv.merge(
        table_rows, how="left", on=["schema_name", "table_name"]
    )

    # if an oracle database is analysed, final table will have 9 columns, for other databases table info will have 
    # 8 columns
    if len(table_csv.columns) == 8:

        table_csv.columns = [
            "schema_name",
            "table_name",
            "nb_columns",
            "keys",
            "keys_columns",
            "target_table",
            "nb_rows",
            "table_comments",
        ]
        cols = [
            "schema_name",
            "table_name",
            "nb_columns",
            "nb_rows",
            "keys",
            "keys_columns",
            "target_table",
           "table_comments"
        ]
        
        table_csv = table_csv[cols]
    elif len(table_csv.columns) == 9:

        table_csv.columns = [
            "schema_name",
            "table_name",
            "nb_columns",
            "keys",
            "keys2",
            "keys_columns",
            "target_table",
            "nb_rows",
            "table_comments",
        ]
        cols = [
            "schema_name",
            "table_name",
            "nb_columns",
            "nb_rows",
            "keys",
            "keys2",
            "keys_columns",
            "target_table",
            "table_comments",
        ]
        table_csv=table_csv[cols]

    return table_csv

In [26]:
table_csv = table_data(columns_csv, df_table_pk)

In [27]:
# Save to CSV
table_csv.to_csv('tables_info.csv', index = False)

## 4) First impression DB

### Number of tables and their relations

In [28]:
#Number of schemas/owners
print("Number of schemas =", df_table.schema_name.nunique())
# Number of tables
print("Number of tables =", df_table.table_name.nunique())
# Number of unique columns names
print("Number of unique columns = ", df_table.column_name.nunique())
# Number of columns (including keys)
print("Number of columns = ", df_table.column_name.count())

Number of schemas = 1
Number of tables = 234
Number of unique columns =  169
Number of columns =  3435


In [29]:
stats = pd.DataFrame(np.array([['N° Schemas', df_table.schema_name.nunique()],['N° Tables', df_table.table_name.nunique()],['N° unique columns', df_table.column_name.nunique()],['N° columns', df_table.column_name.count()]]),columns = ['','Count'])


#### Fig 3. Relative amount of data per table
    Each big square represents a table, inside each of the columns and if the columns are important keys.
    Each a column is a key, then the target table is indicated.
    Color scaled indicators of column filling in percentage (amount of not-null data / total number of rows).
    Uncomment if needed.

In [30]:
#fig_3 = px.treemap(
#    columns_csv,
#    path=["schema_name", "table_name", "column_name", "key2"],
#    color="percentage",
#    color_continuous_scale="RdBu",
#    color_continuous_midpoint=50,
#)
#
#fig_3.show()

## Fig5. Interactive Dashboard

The reconstruction of the relationships is done with all the columns or the primary/foreign keys, according to the chose (primary/all).
Relationship is reconstructed usign Networkx and interactively ploted in Dash/Plotly.

In [31]:
G = nx.from_pandas_edgelist(df_table_pk2, source="from", target="target", edge_attr=True)

pos = nx.layout.spring_layout(G)

elements = nx.cytoscape_data(G)

nodes = elements["elements"]["nodes"]

palette = sns.color_palette(
    None, len(nodes)
)  # TODO Change color for schema and not nodes!

col_swatch = list(palette.as_hex())


for i in range(len(nodes)):
    a = nodes[i]["data"]["id"]
    b = {"position": {"x": 1000 * pos[a][0], "y": 1000 * pos[a][1]}}
    nodes[i].update(b)
for i in range(len(nodes)):
    c = {"selectable": True}
    nodes[i]["data"].update(c)
edges = elements["elements"]["edges"]

for i in range(len(nodes)):
    d = {"color_node": col_swatch[i], "size": 20}
    nodes[i]["data"].update(d)

elements_dash = nodes + edges

In [32]:
stylesheets = [
    {
        "selector": "edge",
        "style": {"color": "data(keys)", "line-color": "lightgray", "width": 3},
    },
    {
        "selector": "node",
        "style": {
            "label": "data(id)",
            "font-size": 20,
            "background-color": "data(color_node)",
            "line-color": "data(color_node)",
            "width": "data(size)",
            "height": "data(size)",
        },
    },
]

In [33]:
styles = {"pre": {"border": "thin lightgrey solid", "overflowX": "scroll"}}

In [34]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SPACELAB])

#### LAYOUT DASHBOARD ####


app.layout = html.Div(
    [
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(
                            [
                                html.H1(
                                    "Data Base exploration",
                                    className="bg-primary text-white text-center",
                                )
                            ]
                        )
                    ]
                )
            ]
        ),
        html.Div([dbc.Row([html.Br([])])]),
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(width=1),
                        dbc.Col(
                            dbc.Container(
                                [
                                    dbc.Row(
                                        [html.H5("Schema:", className="text-info")]
                                    ),
                                    dbc.Row(
                                        [
                                            dcc.Dropdown(
                                                id="dpdn5",
                                                multi=True,
                                                clearable=True,
                                                options=[
                                                    {
                                                        "label": name,
                                                        "value": name,
                                                    }
                                                    for name in list(
                                                        columns_csv[
                                                            "schema_name"
                                                        ].unique()
                                                    )
                                                ],
                                                style=dict(width="100%"),
                                            )
                                        ]
                                    ),
                                    dbc.Row([html.H5("Table:", className="text-info")]),
                                    dbc.Row(
                                        [
                                            dcc.Dropdown(
                                                id="dpdn6",
                                                multi=True,
                                                clearable=True,
                                                options=[
                                                    {
                                                        "label": name,
                                                        "value": name,
                                                    }
                                                    for name in list(
                                                        columns_csv[
                                                            "table_name"
                                                        ].unique()
                                                    )
                                                ],
                                                style=dict(width="100%"),
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        [
                                            html.H5(
                                                "Select Network Layout:",
                                                className="text-info",
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        [
                                            dcc.Dropdown(
                                                id="dpdn1",
                                                value="preset",
                                                clearable=False,
                                                options=[
                                                    {
                                                        "label": name.capitalize(),
                                                        "value": name,
                                                    }
                                                    for name in [
                                                        "breadthfirst",
                                                        "grid",
                                                        "random",
                                                        "circle",
                                                        "cose",
                                                        "concentric",
                                                    ]
                                                ],
                                                style=dict(width="100%"),
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        [
                                            html.H5(
                                                "Select Neighbors Distance:",
                                                className="text-info",
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        [
                                            dcc.Dropdown(
                                                id="dpdn2",
                                                value=1,
                                                clearable=False,
                                                options=[
                                                    {
                                                        "label": name,
                                                        "value": name,
                                                    }
                                                    for name in [
                                                        1,
                                                        2,
                                                        3,
                                                    ]
                                                ],
                                                style=dict(width="100%"),
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        [
                                            html.Button(
                                                "Update",
                                                id="update-button",
                                                className="mr-3",
                                            )
                                        ]
                                    ),
                                ]
                            ),
                            width=3,
                        ),
                        dbc.Col(
                            [
                                dbc.Container(
                                    cyto.Cytoscape(
                                        id="cytoscape-node",
                                        layout={"name": "preset"},
                                        style={
                                            "width": "100%",
                                            "height": "500px",
                                        },
                                        elements=elements_dash,
                                        stylesheet=stylesheets,
                                    )
                                )
                            ],
                            width=5,
                        ),
                        dbc.Col(
                            dbc.Container(
                                [
                                    dbc.Row(
                                        [
                                            html.H5(
                                                "Click on a connector in the network to see description:",
                                                className="text-info",
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        [
                                            html.Pre(
                                                id="cytoscape-tapEdgeData-json",
                                                style=styles["pre"],
                                            )
                                        ]
                                    ),
                                ]
                            ),
                            width=2,
                        ),
                        dbc.Col(width=1),
                    ]
                )
            ]
        ),
        html.Div([dbc.Row([html.Br([])])]),
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(width=1),
                        dbc.Col(
                            dbc.Container(
                                [
                                    dbc.Row(
                                        [
                                            html.H5(
                                                "Database size:", className="text-info"
                                            )
                                        ]
                                    ),
                                    dbc.Row(
                                        [
                                            dash_table.DataTable(
                                                id="table2",
                                                columns=[
                                                    {"name": i, "id": i}
                                                    for i in stats.columns
                                                ],
                                                data=stats.to_dict("records"),
                                            ),
                                        ]
                                    ),
                                ]
                            ),
                            width=2,
                        ),
                        dbc.Col(width=1),
                        dbc.Col(
                            dbc.Container([dcc.Graph(id="my-graph")]),
                            width=8,
                        ),
                    ]
                )
            ]
        ),
        html.Div([dbc.Row([html.Br([])])]),
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(width=5),
                        dbc.Col(
                            html.H5(
                                "Table Preview",
                                className="text-info",
                            )
                        ),
                    ]
                )
            ]
        ),
        html.Div([dbc.Row([html.Br([])])]),
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(width=1),
                        dbc.Col(
                            html.Table(
                                id="table",
                                className="dbc_light",
                            ),
                            width=8,
                        ),
                        dbc.Col(width=1),
                    ]
                )
            ]
        ),
        html.Div([dbc.Row([html.Br([])])]),
    ],
)


# Schema selection



# Table selection


@app.callback(Output("dpdn6", "options"), [Input("dpdn5", "value")])
def update_tables(selected_schema):

    if selected_schema == None:
        df = columns_csv
        return [
            {"label": name, "value": name} for name in list(df["table_name"].unique())
        ]

    else:
        df = df_table_pk2[df_table_pk2["schema"].isin(selected_schema)]
        return [{"label": name, "value": name} for name in list(df["from"].unique())]


# -  Network Format using the dropdown menu


@app.callback(Output("cytoscape-node", "layout"), Input("dpdn1", "value"))
def update_layout(layout_value):
    if layout_value == "breadthfirst":
        return {"name": layout_value, "animate": True}
    else:
        return {"name": layout_value, "animate": True}


# Network Neighbors degrees


@app.callback(
    Output("cytoscape-node", "elements"),
    [Input("update-button", "n_clicks")],
    [
        State("cytoscape-node", "elements"),
        State("dpdn6", "value"),
        State("dpdn2", "value"),
    ],
)
def keep_nodes(_, elements, data, relation_value=1):
    if data is None:
        return no_update
    else:

        radius = relation_value
        D = ego_graph(
            G, data[0], radius=radius, center=True, undirected=False, distance=None
        )
        elements = nx.cytoscape_data(D)
        nodes = elements["elements"]["nodes"]

        for i in range(len(nodes)):
            a = nodes[i]["data"]["id"]
            b = {"position": {"x": 1000 * pos[a][0], "y": 1000 * pos[a][1]}}
            nodes[i].update(b)
        for i in range(len(nodes)):
            c = {"selectable": True}
            nodes[i]["data"].update(c)
        edges = elements["elements"]["edges"]
        for i in range(len(nodes)):
            d = {"color_node": col_swatch[i]}
            nodes[i]["data"].update(d)
        edges = elements["elements"]["edges"]
        elements_dash = nodes + edges
        return elements_dash
    return elements


@app.callback(
    Output("cytoscape-tapEdgeData-json", "children"),
    Input("cytoscape-node", "tapEdgeData"),
)
def displayTapNodeData(data):
    return json.dumps(data, indent=2)


@app.callback(Output("my-graph", "figure"), Input("dpdn6", "value"))
def update_nodes(data):

    if data is None:
        return no_update

    else:
        label = data[0]
        dff = columns_csv[columns_csv["table_name"] == label]
        fig = px.bar(
            dff,
            x="column_name",
            y="percentage",
            title="Percentage of filled data per column in: {}".format(label),
            color_discrete_sequence=["indianred", "RebeccaPurple", "darkgreen"],
        )
        fig.update_xaxes(categoryorder="total descending")
        fig.add_hline(y=100)
        fig.update_layout(template="plotly_white", showlegend=False)
        fig.update_traces(marker_line_color="RebeccaPurple", marker_line_width=1.5)
        return fig


# - Show first 10 lines of selected table


@app.callback(
    Output("table", "children"),
    [Input("update-button", "n_clicks")],
    [State("dpdn5", "value"), State("dpdn6", "value")],
)
def table(_, schema_name, table_name):

    if schema_name is None:
        return no_update
    if table_name is None:
        return no_update

    else:
        query4 = query4_first10.format(schema_name[0], table_name[0])
        df = query_wo_header(query4)

        table = dash_table.DataTable(
            columns=[{"name": i, "id": i} for i in sorted(df.columns)],
            data=df.to_dict("records"),
            style_data={
                "whiteSpace": "normal",
                "height": "auto",
            },
            style_table={"overflowX": "auto", "width": "1500px"},
            fill_width=False
        )
        return table


app.run_server(port=8081)

Dash app running on http://127.0.0.1:8081/
